## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-18-07-11-26 +0000,bbc,UK inflation falls to 3% in January on lower f...,https://www.bbc.com/news/articles/c5yg04jg1n5o...
1,2026-02-18-07-09-00 +0000,wsj,U.K. Inflation Slowed in January,https://www.wsj.com/economy/u-k-inflation-slow...
2,2026-02-18-07-07-38 +0000,nypost,Mamdani’s ‘cuckoo’ property tax hike will land...,https://nypost.com/2026/02/18/us-news/mamdanis...
3,2026-02-18-06-55-41 +0000,bbc,Peru's Congress ousts president after four mon...,https://www.bbc.com/news/articles/cz7gvdx457vo...
4,2026-02-18-06-55-00 +0000,wsj,U.S. 10-Year Treasury Yield ‘Looks Good’ at 4.20%,https://www.wsj.com/finance/investing/jgb-pric...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2460/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
62,jackson,28
152,trump,26
61,jesse,24
118,talks,16
159,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
230,2026-02-17-18-23-29 +0000,nyt,"Al Sharpton, Bernice King and Trump Among Trib...",https://www.nytimes.com/2026/02/17/us/politics...,87
170,2026-02-17-21-36-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...,86
101,2026-02-18-00-23-43 +0000,bbc,'He did it for us': US soldier recalls Jesse J...,https://www.bbc.com/news/articles/cwykn2endnyo...,81
88,2026-02-18-00-48-22 +0000,bbc,"Under pressure from Trump, Venezuela's new pre...",https://www.bbc.com/news/articles/cn87rv0jdy1o...,79
330,2026-02-17-11-14-34 +0000,latimes,"Jesse Jackson, one of the nation's most powerf...",https://www.latimes.com/obituaries/story/2026-...,79


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
230,87,2026-02-17-18-23-29 +0000,nyt,"Al Sharpton, Bernice King and Trump Among Trib...",https://www.nytimes.com/2026/02/17/us/politics...
310,71,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...
170,60,2026-02-17-21-36-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...
27,55,2026-02-18-04-50-10 +0000,bbc,Suspect in Rhode Island ice hockey shooting ki...,https://www.bbc.com/news/articles/cy4wvk9vn40o...
245,50,2026-02-17-17-49-41 +0000,nypost,Stephen Colbert rips CBS for pulling interview...,https://nypost.com/2026/02/17/media/colbert-ri...
2,49,2026-02-18-07-07-38 +0000,nypost,Mamdani’s ‘cuckoo’ property tax hike will land...,https://nypost.com/2026/02/18/us-news/mamdanis...
235,45,2026-02-17-18-14-06 +0000,nyt,Ukraine and Russia Hold New Round of Peace Tal...,https://www.nytimes.com/2026/02/17/world/europ...
277,45,2026-02-17-16-02-41 +0000,nypost,"Iran, US agree on ‘guiding principles’ of nucl...",https://nypost.com/2026/02/17/world-news/iran-...
354,38,2026-02-17-09-54-34 +0000,bbc,Long queues and confusion mar first day of Ind...,https://www.bbc.com/news/articles/ceqvjgrvpn3o...
33,35,2026-02-18-04-04-52 +0000,latimes,"UCLA fires top finance officer, saying he made...",https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
